In [1]:
from src.midi.midiToNotes import MidiProcessor
from src.piano.piano import Piano
from src.hand.hand import Hand
from src.hand.finger import Finger
from src.recorder.recorder import Recorder
from src.recorder.recorderPool import RecorderPool

midi_name = "诀别书"
track_numbers = [2]
channel_number = 15
hand_range = 12
finger_range = 4
FPS = 60
# 是否要提高八度
higher_octave = False

# 初始各种文件路径
track_text = str(track_numbers[0]) if len(track_numbers) == 1 else "_".join([str(track_number) for track_number in track_numbers])
hand_recorder_file_path = f"output/hand_recorders/{midi_name}_{track_text}.hand"

# 读取midi文件，获得音高和时间轴
midi_processor = MidiProcessor(midi_name, track_numbers, channel_number=channel_number,FPS=FPS)
midi_info = midi_processor.export_midi_info()
print(midi_info)
notes_maps = midi_processor.generate_notes_map_and_messages(higher_octave)

# 初始化钢琴
piano = Piano()

# 初始化左右手,左手放在C3，右手放在C5
left_hand = Hand([Finger(0, piano.note_to_key(48)), Finger(1, piano.note_to_key(50)), Finger(2, piano.note_to_key(52)), Finger(3, piano.note_to_key(53)), Finger(4, piano.note_to_key(55))], piano, True)
right_hand = Hand([Finger(5, piano.note_to_key(72), False), Finger(6, piano.note_to_key(74), False), Finger(
    7, piano.note_to_key(76), False), Finger(8, piano.note_to_key(77), False), Finger(9, piano.note_to_key(79), False)], piano, False)

init_real_tick = 0.0
init_real_ticks = []

# 初始化recorder
recorder = Recorder(piano, [left_hand], [right_hand],
                    0, init_real_tick, init_real_ticks)

# 初始化recorderPool
pool_size = 100
recorder_pool = RecorderPool([recorder],pool_size,0)

Track 0: 
Track 1: 
Track 2: MIDITrack
Track 2, Channel 15: Acoustic Grand Piano
Track 2, Channel 15: Acoustic Grand Piano
Track 2, Channel 15: Acoustic Grand Piano
Track 2, Channel 15: Acoustic Grand Piano
Track 2, Channel 15: Acoustic Grand Piano

一共有1个轨道
pitch_wheel_map 保存到了 asset/temp/pitch_wheel_map.json
messages 保存到了 asset/temp/messages.json
全曲的速度变化是:
在1轨，tick为0时，速度变为500000
在1轨，tick为0时，速度变为500000
在1轨，tick为0时，速度变为500000

全曲的每拍tick数是:96

总共有 989 个和弦/单音符
总共按下了 1551 个音符（包含和弦中的每个音符）
其中不重复的音符有 48 个
如果以60的fps做成动画，一共是41581 ticks, 合计12994.0625帧, 约216.56770833333334秒
notes_map 保存到了 asset/temp/notes_map.json


In [2]:
from tqdm import tqdm

# 使用notes_map，开始迭代更新recorder，并且放入recorder_pool中
for notes_map in tqdm(notes_maps, desc="生成指法中……", unit="step"):
    recorder_pool.update_recorder_pool(notes_map, hand_range, finger_range)

recorder_pool.export_pool_info(hand_recorder_file_path)

生成指法中……:  46%|████▌     | 456/989 [00:23<00:19, 27.27step/s]

警告：没能生成新的记录器，当前frame为：6963.4375,当前音符为：[43, 58, 76, 88]
只保留原最佳记录，并且更新frame


生成指法中……: 100%|██████████| 989/989 [00:54<00:00, 18.11step/s]


最优记录的熵值为：28122
已保存至output/hand_recorders/诀别书_2.hand


In [3]:
from src.animation.animator import Animator

avatar_name = 'Kinich'
avatar_info_path = f'asset/avatars/{avatar_name}.avatar'

animator = Animator(hand_recorder_file_path, avatar_info_path,piano)
animator.load_base_state_info()
animator.generate_animation_info()

已加载output/hand_recorders/诀别书_2.hand 和 asset/avatars/Kinich.avatar
动画数据已保存到output/animation_recorders/诀别书_2_Kinich.animation
